# VGG-16

<img src="img/vgg-16.png">

Gambar diatas adalah arsitektur dari VGG-16 (16 Layer). Yang akan kita gunakan adalah Feature Extraction Layer saja, tentu saja dengan weights yang dapat kita download. Weights nya berupa file .h5 seperti yang sudah kita gunakan sebelumnya.

**FC layer pada VGG-16 terdiri dari 4096–4096–4096 neuron pada hidden layer** dan **1000 neuron pada output layer**, karena **ImageNet mempunyai 1000 class**. Sedangkan dataset kita hanya mempunyai **2 class (Male/Female)**, sehingga kita harus membuat FC layer versi kita sendiri.

**Kita akan gunakan FC Layer KorNet** yaitu **32 neuron pada hidden layer dan 1 neuron pada output layer** dengan **sigmoid activation**.

# VGG-16 Dependencies and Variable

Hampir sama dengan dependencies KorNet, namun kita membutuhkan package applications untuk dapat menggunakan VGG-16.

In [4]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

# Images Dimensions
img_width, img_height = 128, 128

train_data_dir = 'data/datasets/train'
validation_data_dir = 'data/datasets/validation'
nb_train_samples = 800
nb_validation_samples = 240
epochs = 50
batch_size = 16

# TensorBoard Callbacks
callbacks = TensorBoard(log_dir='./graph')

# VGG-16 Model and Data Augmentation

In [2]:
# Build VGG16
model = applications.VGG16(include_top=False, weights='imagenet')

# Training Data Augmentation
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Rescale Testing Data
test_datagen = ImageDataGenerator(rescale=1. / 255)

# Train Data Generator
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

train_features = model.predict_generator(train_generator, nb_train_samples // batch_size, verbose=1)

np.save('train_features.npy', train_features)

# Testing Data Generator
validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                        target_size=(img_width, img_height),
                                                        batch_size=batch_size,
                                                        class_mode='binary')

validation_features = model.predict_generator(validation_generator, nb_train_samples // batch_size, verbose=1)

np.save('val_features.npy', validation_features)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Found 800 images belonging to 2 classes.

50/50 [==============================] - 68s 1s/step
Found 240 images belonging to 2 classes.
50/50 [==============================] - 69s 1s/step


Dengan menggunakan package applications dari Keras. Kita bisa langsung menggunakan VGG-16 tanpa harus menyusunnya layer demi layer dan mendownload **weights** nya.

Argument `include_top=False` diatas menandakan jika kita **tidak menggunakan FC Layer dari VGG-16**. Sehingga jika kita melakukan "predict" untuk model ini maka yang akan terjadi adalah dataset akan mengalir pada feature extraction layer dari VGG-16. Hasilnya adalah **feature map yang bisa kita simpan pada file `train_features.npy` dan `val_features.npy`** yang nantinya bisa kita flatten dan kita gunakan untuk melakukan training pada FC Layer versi kita sendiri.

# VGG-16 FC Layer

In [5]:
# Build Train Data
train_data = np.load('train_features.npy')
train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

# Build Validation Data
validation_data = np.load('val_features.npy')
validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

# Build FC Layer
fc_model = Sequential()
fc_model.add(Flatten(input_shape=train_data.shape[1:]))
fc_model.add(Dense(32, activation='relu'))
fc_model.add(Dense(1, activation='sigmoid'))

# Adam Optimizer and Cross Entropy Loss
adam = Adam(lr=0.0001)
fc_model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

fc_model.fit(train_data, train_labels,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(validation_data, validation_labels), 
            callbacks=[callbacks])
            
fc_model.save_weights('vgg16.py')

ValueError: Input arrays should have the same number of samples as target arrays. Found 800 input samples and 240 target samples.

Setelah kita mendapatkan **feature map pada tahap sebelumnya**, kita akan **gunakan feature map tersebut** sebagai **data training** dan **testing** untuk FC Layer kita.

<img src="img/tf_board_vgg-16.png">

Namun jika dilihat dari grafiknya, masih terjadi overfitting. Kita bisa tangani ini dengan cara menggunakan **FC Layer yang lebih sederhana**, **menggunakan Dropout**, **melakukan augmentasi yang lebih agresif lagi** atau **menambah jumlah data**.